<a href="https://colab.research.google.com/github/ChinaStark/-operating-system_practice/blob/main/nb/Qwen3_(4B)-GRPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

Unsloth now supports Text-to-Speech (TTS) models. Read our [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning).

Read our **[Qwen3 Guide](https://docs.unsloth.ai/basics/qwen3-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm==0.8.5.post1

In [2]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm==0.8.5.post1
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

### Unsloth

Goal: To convert `Qwen3-4B-Base` into a reasoning model via GRPO by using OpenR1's Math dataset.

We first pre fine-tune the model to make GRPO skip trying to match formatting - this speeds GRPO up.

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-4B-Base",
    max_seq_length = max_seq_length,
    load_in_4bit = False, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.7, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank*2, # *2 speeds up training
    use_gradient_checkpointing = "unsloth", # Reduces memory usage
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 06-18 08:29:19 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 06-18 08:29:19 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.6.2: Fast Qwen3 patching. Transformers: 4.52.4. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/Qwen3-4B-Base with actual GPU utilization = 69.34%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 2048

tokenizer_config.json:   0%|          | 0.00/5.43k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Unsloth 2025.6.2 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


### GRPO chat template
Since we're using a base model, we should set a chat template. You can make your own chat template as well!
1. DeepSeek uses `<think>` and `</think>`, but this is **not** necessary - you can customize it however you like!
2. A `system_prompt` is recommended to at least guide the model's responses.

In [4]:
reasoning_start = "<start_working_out>" # Acts as <think>
reasoning_end   = "<end_working_out>"   # Acts as </think>
solution_start  = "<SOLUTION>"
solution_end    = "</SOLUTION>"

system_prompt = \
f"""You are a very professional SQL expert. Now you are given a question and the corresponding schema in the following text:
### schema
{{SCHEMA}}
Think about the problem and provide your working out.
Place it between {reasoning_start} and {reasoning_end}.
Then, provide your solution between {solution_start}{solution_end}"""
system_prompt

'You are a very professional SQL expert. Now you are given a question and the corresponding schema in the following text:\n### schema\n{SCHEMA}\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION>'

We create a simple chat template below. Notice `add_generation_prompt` includes prepending `<start_working_out>` to guide the model to start its reasoning process.

In [5]:
chat_template = \
    "{% if messages[0]['role'] == 'system' %}"\
        "{{ messages[0]['content'] + eos_token }}"\
        "{% set loop_messages = messages[1:] %}"\
    "{% else %}"\
        "{{ '{system_prompt}' + eos_token }}"\
        "{% set loop_messages = messages %}"\
    "{% endif %}"\
    "{% for message in loop_messages %}"\
        "{% if message['role'] == 'user' %}"\
            "{{ message['content'] }}"\
        "{% elif message['role'] == 'assistant' %}"\
            "{{ message['content'] + eos_token }}"\
        "{% endif %}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}{{ '{reasoning_start}' }}"\
    "{% endif %}"

# Replace with out specific template:
chat_template = chat_template\
    .replace("'{system_prompt}'",   f"'{system_prompt}'")\
    .replace("'{reasoning_start}'", f"'{reasoning_start}'")
tokenizer.chat_template = chat_template

In [6]:
chat_template

"{% if messages[0]['role'] == 'system' %}{{ messages[0]['content'] + eos_token }}{% set loop_messages = messages[1:] %}{% else %}{{ 'You are a very professional SQL expert. Now you are given a question and the corresponding schema in the following text:\n### schema\n{SCHEMA}\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION>' + eos_token }}{% set loop_messages = messages %}{% endif %}{% for message in loop_messages %}{% if message['role'] == 'user' %}{{ message['content'] }}{% elif message['role'] == 'assistant' %}{{ message['content'] + eos_token }}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ '<start_working_out>' }}{% endif %}"

Let's see how our chat template behaves on an example:

In [7]:
tokenizer.apply_chat_template([
    {"role" : "user", "content" : "What is 1+1?"},
    {"role" : "assistant", "content" : f"{reasoning_start}I think it's 2.{reasoning_end}{solution_start}2{solution_end}"},
    {"role" : "user", "content" : "What is 2+2?"},
], tokenize = False, add_generation_prompt = True)

"You are a very professional SQL expert. Now you are given a question and the corresponding schema in the following text:\n### schema\n{SCHEMA}\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION><|endoftext|>What is 1+1?<start_working_out>I think it's 2.<end_working_out><SOLUTION>2</SOLUTION><|endoftext|>What is 2+2?<start_working_out>"

### Pre fine-tuning for formatting
We now use a subset of NVIDIA's [Open Math Reasoning dataset](https://huggingface.co/datasets/nvidia/OpenMathReasoning) which was filtered to only include high quality DeepSeek R1 traces.

We'll only filter ~59 or so examples to first "prime" / pre fine-tune the model to understand our custom GRPO formatting.

In [9]:
from datasets import load_dataset
import pandas as pd
import numpy as np

dataset = load_dataset("json", data_files="/content/reasoned_bird_train.json", split="train")
dataset = dataset.to_pandas()[
    ["evidence", "inputtext", "text","sql","reason_text", "question"]
]


dataset

Generating train split: 0 examples [00:00, ? examples/s]

,evidence,inputtext,text,sql,reason_text,question
0,released in the year 1945 refers to movie_rele...,"table lists , columns = [ lists.user_id ( inte...",released in the year 1945 refers to movie_rele...,SELECT movie_title FROM movies WHERE movie_rel...,**Step One: Problem Decomposition and Planning...,Name movie titles released in year 1945. Sort ...
1,most popular movie refers to MAX(movie_popular...,"table lists , columns = [ lists.user_id ( inte...",most popular movie refers to MAX(movie_popular...,"SELECT movie_title, movie_release_year, direct...",**Step One: Problem Decomposition and Planning...,State the most popular movie? When was it rele...
2,longest movie title refers to MAX(LENGTH(movie...,"table lists , columns = [ lists.user_id ( inte...",longest movie title refers to MAX(LENGTH(movie...,"SELECT movie_title, movie_release_year FROM mo...",**Step One: Problem Decomposition and Planning...,What is the name of the longest movie title? W...
3,movie with the most rating refers to MAX(SUM(r...,"table lists , columns = [ lists.user_id ( inte...",movie with the most rating refers to MAX(SUM(r...,SELECT movie_title FROM movies GROUP BY movie_...,**Step One: Problem Decomposition and Planning...,Name the movie with the most ratings.
4,average = AVG(movie_popularity); number of Mub...,"table lists , columns = [ lists.user_id ( inte...",average = AVG(movie_popularity); number of Mub...,SELECT avg(movie_popularity) FROM movies WHERE...,**Step One: Problem Decomposition and Planning...,What is the average number of Mubi users who l...
...,...,...,...,...,...,...
83,Innocence Unprotected' and 'When Will I Be Lov...,"table lists , columns = [ lists.user_id ( inte...",Innocence Unprotected' and 'When Will I Be Lov...,SELECT sum(CASE WHEN movies.movie_title = 'Inn...,**Step One: Problem Decomposition and Planning...,How much higher is the average rating score of...
84,"Tokyo Eyes' is movie_title, director refers to...","table lists , columns = [ lists.user_id ( inte...","Tokyo Eyes' is movie_title, director refers to...",SELECT director_name FROM movies WHERE movie_t...,**Step One: Problem Decomposition and Planning...,"Who was the director of the movie ""Tokyo Eyes""？"
85,film released in 2007 refers to movie_release_...,"table lists , columns = [ lists.user_id ( inte...",film released in 2007 refers to movie_release_...,SELECT count(*) FROM movies WHERE movie_releas...,**Step One: Problem Decomposition and Planning...,How many films were released in 2007?
86,released in 2006 refers to movie_release_year ...,"table lists , columns = [ lists.user_id ( inte...",released in 2006 refers to movie_release_year ...,SELECT movie_title FROM movies WHERE movie_rel...,**Step One: Problem Decomposition and Planning...,Which of the films released in 2006 was the mo...


In [10]:
dataset.head(1)['text'].tolist()[0]

'released in the year 1945 refers to movie_release_year = 1945; Name movie titles released in year 1945. Sort the listing by the descending order of movie popularity.'

We have to format the dataset to follow our GRPO style formatting:

In [11]:
def format_dataset(x):
    expected_answer = x["sql"]
    problem = x["question"]
    if len(x['evidence']):problem = x["question"] + " and there have some extra knowledge: " + x['evidence']

    # Remove generated <think> and </think>
    thoughts = x["reason_text"]
    schema = x['inputtext']
    # Strip newlines on left and right
    thoughts = thoughts.strip()
    # Add our custom formatting
    final_prompt = \
        reasoning_start + thoughts + reasoning_end + \
        solution_start + expected_answer + solution_end
    return [
        {"role" : "system",    "content" : system_prompt.replace("{SCHEMA}", schema)},
        {"role" : "user",      "content" : problem},
        {"role" : "assistant", "content" : final_prompt},
    ]

dataset["Messages"] = dataset.apply(format_dataset, axis = 1)

In [12]:
system_prompt

'You are a very professional SQL expert. Now you are given a question and the corresponding schema in the following text:\n### schema\n{SCHEMA}\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION>'

Check to see if it worked:

In [13]:
dataset["Messages"][0]

[{'role': 'system',
  'content': 'You are a very professional SQL expert. Now you are given a question and the corresponding schema in the following text:\n### schema\ntable lists , columns = [ lists.user_id ( integer | values : 88260493, 45204418 ), lists.list_id ( integer | primary key | values : 1, 2 ), lists.list_title ( text | values : Headscratchers ), lists.list_movie_number ( integer | values : 5, 3 ), lists.list_update_timestamp_utc ( text | values : 2019-01-24 19:16:18, 2018-12-03 15:12:20 ), lists.list_creation_timestamp_utc ( text | values : 2009-11-11 00:02:21, 2009-11-11 00:05:11 ), lists.list_followers ( integer | values : 5, 1 ), lists.list_url ( text ), lists.list_comments ( integer | values : 3, 2 ), lists.list_description ( text ), lists.list_cover_image_url ( text ), lists.list_first_image_url ( text ), lists.list_second_image_url ( text ), lists.list_third_image_url ( text ) ]\ntable movies , columns = [ movies.movie_id ( integer | primary key | values : 1, 2 ), mo

In [14]:
tokenizer.apply_chat_template(dataset["Messages"][0], tokenize = False)

"You are a very professional SQL expert. Now you are given a question and the corresponding schema in the following text:\n### schema\ntable lists , columns = [ lists.user_id ( integer | values : 88260493, 45204418 ), lists.list_id ( integer | primary key | values : 1, 2 ), lists.list_title ( text | values : Headscratchers ), lists.list_movie_number ( integer | values : 5, 3 ), lists.list_update_timestamp_utc ( text | values : 2019-01-24 19:16:18, 2018-12-03 15:12:20 ), lists.list_creation_timestamp_utc ( text | values : 2009-11-11 00:02:21, 2009-11-11 00:05:11 ), lists.list_followers ( integer | values : 5, 1 ), lists.list_url ( text ), lists.list_comments ( integer | values : 3, 2 ), lists.list_description ( text ), lists.list_cover_image_url ( text ), lists.list_first_image_url ( text ), lists.list_second_image_url ( text ), lists.list_third_image_url ( text ) ]\ntable movies , columns = [ movies.movie_id ( integer | primary key | values : 1, 2 ), movies.movie_title ( text | values 

Let's truncate the pre fine-tuning dataset to `max_seq_length/2` since we don't want too long reasoning traces.

Note this might take 2 minutes!

In [15]:
dataset["N"] = dataset["Messages"].apply(lambda x: len(tokenizer.apply_chat_template(x)))



In [16]:
max(dataset['N'])

2357

In [17]:
dataset.loc[dataset["N"] <= max_seq_length]

,evidence,inputtext,text,sql,reason_text,question,Messages,N
0,released in the year 1945 refers to movie_rele...,"table lists , columns = [ lists.user_id ( inte...",released in the year 1945 refers to movie_rele...,SELECT movie_title FROM movies WHERE movie_rel...,**Step One: Problem Decomposition and Planning...,Name movie titles released in year 1945. Sort ...,"[{'role': 'system', 'content': 'You are a very...",1927
1,most popular movie refers to MAX(movie_popular...,"table lists , columns = [ lists.user_id ( inte...",most popular movie refers to MAX(movie_popular...,"SELECT movie_title, movie_release_year, direct...",**Step One: Problem Decomposition and Planning...,State the most popular movie? When was it rele...,"[{'role': 'system', 'content': 'You are a very...",1909
2,longest movie title refers to MAX(LENGTH(movie...,"table lists , columns = [ lists.user_id ( inte...",longest movie title refers to MAX(LENGTH(movie...,"SELECT movie_title, movie_release_year FROM mo...",**Step One: Problem Decomposition and Planning...,What is the name of the longest movie title? W...,"[{'role': 'system', 'content': 'You are a very...",1798
3,movie with the most rating refers to MAX(SUM(r...,"table lists , columns = [ lists.user_id ( inte...",movie with the most rating refers to MAX(SUM(r...,SELECT movie_title FROM movies GROUP BY movie_...,**Step One: Problem Decomposition and Planning...,Name the movie with the most ratings.,"[{'role': 'system', 'content': 'You are a very...",1985
4,average = AVG(movie_popularity); number of Mub...,"table lists , columns = [ lists.user_id ( inte...",average = AVG(movie_popularity); number of Mub...,SELECT avg(movie_popularity) FROM movies WHERE...,**Step One: Problem Decomposition and Planning...,What is the average number of Mubi users who l...,"[{'role': 'system', 'content': 'You are a very...",1833
...,...,...,...,...,...,...,...,...
81,A Way of Life' is movie_title; average rating ...,"table lists , columns = [ lists.user_id ( inte...",A Way of Life' is movie_title; average rating ...,SELECT avg(ratings.rating_score) FROM ratings ...,**Step One: Problem Decomposition and Planning...,"What's the average rating score of the movie ""...","[{'role': 'system', 'content': 'You are a very...",1957
84,"Tokyo Eyes' is movie_title, director refers to...","table lists , columns = [ lists.user_id ( inte...","Tokyo Eyes' is movie_title, director refers to...",SELECT director_name FROM movies WHERE movie_t...,**Step One: Problem Decomposition and Planning...,"Who was the director of the movie ""Tokyo Eyes""？","[{'role': 'system', 'content': 'You are a very...",1779
85,film released in 2007 refers to movie_release_...,"table lists , columns = [ lists.user_id ( inte...",film released in 2007 refers to movie_release_...,SELECT count(*) FROM movies WHERE movie_releas...,**Step One: Problem Decomposition and Planning...,How many films were released in 2007?,"[{'role': 'system', 'content': 'You are a very...",1868
86,released in 2006 refers to movie_release_year ...,"table lists , columns = [ lists.user_id ( inte...",released in 2006 refers to movie_release_year ...,SELECT movie_title FROM movies WHERE movie_rel...,**Step One: Problem Decomposition and Planning...,Which of the films released in 2006 was the mo...,"[{'role': 'system', 'content': 'You are a very...",1816


In [18]:
dataset = dataset.loc[dataset["N"] <= max_seq_length].copy()
dataset.shape

(65, 8)

We then tokenize the messages and convert it to a Hugging Face compatible dataset format:

In [19]:
from datasets import Dataset

dataset["text"] = tokenizer.apply_chat_template(dataset["Messages"].values.tolist(), tokenize = False)
dataset = Dataset.from_pandas(dataset)
dataset

Dataset({
    features: ['evidence', 'inputtext', 'text', 'sql', 'reason_text', 'question', 'Messages', 'N', '__index_level_0__'],
    num_rows: 65
})

Let's now pre fine-tune the model so it follows our custom GRPO formatting!

In [20]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 1, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 2, # Set this for 1 full training run.
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 5,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/65 [00:00<?, ? examples/s]

In [21]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 65 | Num Epochs = 2 | Total steps = 130
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 1 x 1) = 1
 "-____-"     Trainable parameters = 66,060,288/4,088,528,384 (1.62% trained)


Step,Training Loss
5,1.001600
10,0.645700
15,0.376800
20,0.228600
25,0.176000
30,0.148300
35,0.130200
40,0.151900
45,0.131300
50,0.133700


Unsloth: Will smartly offload gradients to save VRAM!


TrainOutput(global_step=130, training_loss=0.17833173825190618, metrics={'train_runtime': 431.7703, 'train_samples_per_second': 0.301, 'train_steps_per_second': 0.301, 'total_flos': 5587848685418496.0, 'train_loss': 0.17833173825190618})

Let's check if the model has learnt to follow the custom format:

In [22]:
text = tokenizer.apply_chat_template(
    dataset[0]["Messages"][:2],
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    temperature = 0,
    max_new_tokens = 1024,
    streamer = TextStreamer(tokenizer, skip_prompt = False),
)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


You are a very professional SQL expert. Now you are given a question and the corresponding schema in the following text:
### schema
table lists , columns = [ lists.user_id ( integer | values : 88260493, 45204418 ), lists.list_id ( integer | primary key | values : 1, 2 ), lists.list_title ( text | values : Headscratchers ), lists.list_movie_number ( integer | values : 5, 3 ), lists.list_update_timestamp_utc ( text | values : 2019-01-24 19:16:18, 2018-12-03 15:12:20 ), lists.list_creation_timestamp_utc ( text | values : 2009-11-11 00:02:21, 2009-11-11 00:05:11 ), lists.list_followers ( integer | values : 5, 1 ), lists.list_url ( text ), lists.list_comments ( integer | values : 3, 2 ), lists.list_description ( text ), lists.list_cover_image_url ( text ), lists.list_first_image_url ( text ), lists.list_second_image_url ( text ), lists.list_third_image_url ( text ) ]
table movies , columns = [ movies.movie_id ( integer | primary key | values : 1, 2 ), movies.movie_title ( text | values : La

Yes it did follow the formatting! Great! Let's remove some items before the GRPO step

In [18]:
del dataset
torch.cuda.empty_cache()
import gc
gc.collect()

0

### Data Prep
<a name="Data"></a>

We're using Hugging Face's [Open R1 Math dataset](https://huggingface.co/datasets/open-r1/DAPO-Math-17k-Processed). You can also utilize OpenAI's famous [GSM8K dataset](https://huggingface.co/datasets/openai/gsm8k)

In [19]:
from datasets import load_dataset
dataset = load_dataset("open-r1/DAPO-Math-17k-Processed", "en", split = "train")
dataset

README.md:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

en/train-00000-of-00001.parquet:   0%|          | 0.00/5.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14116 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt', 'solution', 'data_source', 'source_prompt', 'ability', 'reward_model', 'extra_info'],
    num_rows: 14116
})

Let's look at the first row:

In [25]:
dataset[0]

{'evidence': 'released in the year 1945 refers to movie_release_year = 1945;',
 'inputtext': 'table lists , columns = [ lists.user_id ( integer | values : 88260493, 45204418 ), lists.list_id ( integer | primary key | values : 1, 2 ), lists.list_title ( text | values : Headscratchers ), lists.list_movie_number ( integer | values : 5, 3 ), lists.list_update_timestamp_utc ( text | values : 2019-01-24 19:16:18, 2018-12-03 15:12:20 ), lists.list_creation_timestamp_utc ( text | values : 2009-11-11 00:02:21, 2009-11-11 00:05:11 ), lists.list_followers ( integer | values : 5, 1 ), lists.list_url ( text ), lists.list_comments ( integer | values : 3, 2 ), lists.list_description ( text ), lists.list_cover_image_url ( text ), lists.list_first_image_url ( text ), lists.list_second_image_url ( text ), lists.list_third_image_url ( text ) ]\ntable movies , columns = [ movies.movie_id ( integer | primary key | values : 1, 2 ), movies.movie_title ( text | values : La Antena, Elementary Particles ), movi

In [21]:
dataset[0]["solution"]

'34'

In GSM8K, ee notice all answers like about have a ####, so we extract it. But for the Open R1 dataset, we can skip the below.

In [24]:
def extract_hash_answer(text):
    # if "####" not in text: return None
    # return text.split("####")[1].strip()
    return text
extract_hash_answer(dataset[0]["sql"])

'SELECT movie_title FROM movies WHERE movie_release_year = 1945 ORDER BY movie_popularity DESC LIMIT 1'

Let's map the dataset! and see the first row:

In [26]:
dataset = dataset.map(lambda x: {
    "prompt" : [
        {"role": "system", "content": system_prompt.replace("{SCHEMA}", x["inputtext"])},
        {"role": "user",   "content": x["question"]},
    ],
    "answer": x["sql"],
})
dataset[0]

Map:   0%|          | 0/65 [00:00<?, ? examples/s]

{'evidence': 'released in the year 1945 refers to movie_release_year = 1945;',
 'inputtext': 'table lists , columns = [ lists.user_id ( integer | values : 88260493, 45204418 ), lists.list_id ( integer | primary key | values : 1, 2 ), lists.list_title ( text | values : Headscratchers ), lists.list_movie_number ( integer | values : 5, 3 ), lists.list_update_timestamp_utc ( text | values : 2019-01-24 19:16:18, 2018-12-03 15:12:20 ), lists.list_creation_timestamp_utc ( text | values : 2009-11-11 00:02:21, 2009-11-11 00:05:11 ), lists.list_followers ( integer | values : 5, 1 ), lists.list_url ( text ), lists.list_comments ( integer | values : 3, 2 ), lists.list_description ( text ), lists.list_cover_image_url ( text ), lists.list_first_image_url ( text ), lists.list_second_image_url ( text ), lists.list_third_image_url ( text ) ]\ntable movies , columns = [ movies.movie_id ( integer | primary key | values : 1, 2 ), movies.movie_title ( text | values : La Antena, Elementary Particles ), movi

We create a regex format to match the reasoning sections and answers:

In [27]:
import re

# Add optional EOS token matching
solution_end_regex = r"</SOLUTION>[\s]{0,}" + \
    "(?:" + re.escape(tokenizer.eos_token) + ")?"

match_format = re.compile(
    rf"{reasoning_end}.*?"\
    rf"{solution_start}(.+?){solution_end_regex}"\
    rf"[\s]{{0,}}$",
    flags = re.MULTILINE | re.DOTALL
)
match_format

re.compile(r'<end_working_out>.*?<SOLUTION>(.+?)</SOLUTION>[\s]{0,}(?:<\|endoftext\|>)?[\s]{0,}$',
re.MULTILINE|re.DOTALL|re.UNICODE)

We verify it works:

In [29]:
match_format.findall(
    "Let me think!<end_working_out>"\
    f"<SOLUTION>SELECT * from tb_user;</SOLUTION>",
)

['SELECT * from tb_user;']

In [31]:
match_format.findall(
    "<start_working_out>Let me think!<end_working_out>"\
    f"<SOLUTION>  SELECT *   </SOLUTION>\n\n",
)

['  SELECT *   ']

We now want to create a reward function to match the format exactly - we reward it with 3 points if it succeeds:

In [32]:
def match_format_exactly(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Match if format is seen exactly!
        if match_format.search(response) is not None: score += 3.0
        scores.append(score)
    return scores

If it fails, we want to reward the model if it at least follows the format partially, by counting each symbol:

In [33]:
def match_format_approximately(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Count how many keywords are seen - we penalize if too many!
        # If we see 1, then plus some points!

        # No need to reward <start_working_out> since we always prepend it!
        # score += 0.5 if response.count(reasoning_start) == 1 else -1.0
        score += 0.5 if response.count(reasoning_end)   == 1 else -1.0
        score += 0.5 if response.count(solution_start)  == 1 else -1.0
        score += 0.5 if response.count(solution_end)    == 1 else -1.0
        scores.append(score)
    return scores

Finally, we want to extract the generated answer, and reward or penalize it! We also reward it based on how close the answer is to the true one via ratios:

In [35]:
def check_answer(prompts, completions, answer, **kwargs):
    # 取模型的自然语言问题（可选，调试用）
    question = prompts[0][-1]["content"]

    # 提取模型生成的 SQL 响应（不使用```包裹）
    responses = [completion[0]["content"] for completion in completions]

    scores = []
    for pred, true_sql in zip(responses, answer):
        # 规范化：去除多余空白符，并转换为小写
        pred_clean = re.sub(r"\s+", " ", pred.strip().lower())
        true_clean = re.sub(r"\s+", " ", true_sql.strip().lower())

        # Exact Match 计算
        if pred_clean == true_clean:
            scores.append(1.0)
        else:
            scores.append(0.0)
    return scores

Also sometimes it might not be 1 number as the answer, but like a sentence for example "The solution is $20" -> we extract 20.

We also remove possible commas for example as in 123,456

In [30]:
match_numbers = re.compile(
    solution_start + r".*?[\s]{0,}([-]?[\d\.\,]{1,})",
    flags = re.MULTILINE | re.DOTALL
)
print(match_numbers.findall("<SOLUTION>  0.34  </SOLUTION>"))
print(match_numbers.findall("<SOLUTION>  123,456  </SOLUTION>"))
print(match_numbers.findall("<SOLUTION>  -0.234  </SOLUTION>"))
print(match_numbers.findall("<SOLUTION>17</SOLUTION>"))

['0.34']
['123,456']
['-0.234']
['17']


We now prepare our main function which will print out the generated responses and the true answer, along with another reward function which converts text to float via `float` and sees if it's the same.

In [31]:
global PRINTED_TIMES
PRINTED_TIMES = 0
global PRINT_EVERY_STEPS
PRINT_EVERY_STEPS = 5

def check_numbers(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1)
        if (guess := match_numbers.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    # Print only every few steps
    global PRINTED_TIMES
    global PRINT_EVERY_STEPS
    if PRINTED_TIMES % PRINT_EVERY_STEPS == 0:
        print(
            '*'*20 + f"Question:\n{question}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}"
        )
    PRINTED_TIMES += 1

    for guess, true_answer in zip(extracted_responses, answer):
        if guess is None:
            scores.append(-2.5)
            continue
        # Convert to numbers
        try:
            true_answer = float(true_answer.strip())
            # Remove commas like in 123,456
            guess       = float(guess.strip().replace(",", ""))
            scores.append(3.5 if guess == true_answer else -1.5)
        except:
            scores.append(0)
            continue
    return scores

In [36]:
dataset

Dataset({
    features: ['evidence', 'inputtext', 'text', 'sql', 'reason_text', 'question', 'Messages', 'N', '__index_level_0__', 'prompt', 'answer'],
    num_rows: 65
})

Get the top 90% prompt length so we don't accidentally truncate them!

Ie we'll remove the top 10% long prompts.

In [37]:
tokenized = dataset.map(
    lambda x: {"tokens" : tokenizer.apply_chat_template(x["prompt"], add_generation_prompt = True, tokenize = True)},
    batched = True,
)
print(tokenizer.decode(tokenized[0]["tokens"]))
tokenized = tokenized.map(lambda x: {"L" : len(x["tokens"])})

import numpy as np
maximum_length = int(np.quantile(tokenized["L"], 0.9))
print("Max Length = ", maximum_length)

# Filter only samples smaller than 90% max length
dataset = dataset.select(np.where(np.array(tokenized["L"]) <= maximum_length)[0])
del tokenized

Map:   0%|          | 0/65 [00:00<?, ? examples/s]

You are a very professional SQL expert. Now you are given a question and the corresponding schema in the following text:
### schema
table lists , columns = [ lists.user_id ( integer | values : 88260493, 45204418 ), lists.list_id ( integer | primary key | values : 1, 2 ), lists.list_title ( text | values : Headscratchers ), lists.list_movie_number ( integer | values : 5, 3 ), lists.list_update_timestamp_utc ( text | values : 2019-01-24 19:16:18, 2018-12-03 15:12:20 ), lists.list_creation_timestamp_utc ( text | values : 2009-11-11 00:02:21, 2009-11-11 00:05:11 ), lists.list_followers ( integer | values : 5, 1 ), lists.list_url ( text ), lists.list_comments ( integer | values : 3, 2 ), lists.list_description ( text ), lists.list_cover_image_url ( text ), lists.list_first_image_url ( text ), lists.list_second_image_url ( text ), lists.list_third_image_url ( text ) ]
table movies , columns = [ movies.movie_id ( integer | primary key | values : 1, 2 ), movies.movie_title ( text | values : La

Map:   0%|          | 0/65 [00:00<?, ? examples/s]

Max Length =  1206


<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [38]:
max_prompt_length = maximum_length + 1 # + 1 just in case!
max_completion_length = max_seq_length - max_prompt_length

from vllm import SamplingParams
vllm_sampling_params = SamplingParams(
    min_p = 0.1,
    top_p = 1.0,
    top_k = -1,
    seed = 3407,
    stop = [tokenizer.eos_token],
    include_stop_str_in_output = True,
)

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    vllm_sampling_params = vllm_sampling_params,
    temperature = 1.0,
    learning_rate = 5e-6,
    weight_decay = 0.01,
    warmup_ratio = 0.1,
    lr_scheduler_type = "linear",
    optim = "adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 4, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_completion_length,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 100,
    save_steps = 100,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",

    # For optional training + evaluation
    # fp16_full_eval = True,
    # per_device_eval_batch_size = 4,
    # eval_accumulation_steps = 1,
    # eval_strategy = "steps",
    # eval_steps = 1,
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [ ]:
# For optional training + evaluation
# new_dataset = dataset.train_test_split(test_size = 0.01)

trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        match_format_exactly,
        match_format_approximately,
        check_answer,
    ],
    args = training_args,
    train_dataset = dataset,

    # For optional training + evaluation
    # train_dataset = new_dataset["train"],
    # eval_dataset = new_dataset["test"],
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 58 | Num Epochs = 2 | Total steps = 100
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 1 x 1) = 4
 "-____-"     Trainable parameters = 66,060,288/4,088,528,384 (1.62% trained)


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / match_format_exactly / mean,rewards / match_format_exactly / std,rewards / match_format_approximately / mean,rewards / match_format_approximately / std,rewards / check_answer / mean,rewards / check_answer / std
1,0.021200,1.500000,3.674235,810.000000,725.000000,841.000000,0.500000,779.000000,725.000000,833.000000,0.530430,1.500000,1.732051,0.000000,2.121320,0.000000,0.000000
2,0.020200,4.500000,0.000000,717.750000,677.000000,743.000000,0.000000,717.750000,677.000000,743.000000,0.504948,3.000000,0.000000,1.500000,0.000000,0.000000,0.000000
3,0.032600,4.500000,0.000000,681.250000,620.000000,721.000000,0.000000,681.250000,620.000000,721.000000,0.814842,3.000000,0.000000,1.500000,0.000000,0.000000,0.000000
4,0.025800,3.375000,2.250000,729.250000,606.000000,841.000000,0.250000,692.000000,606.000000,738.000000,0.644817,2.250000,1.500000,1.125000,0.750000,0.000000,0.000000
5,0.020200,4.500000,0.000000,637.000000,596.000000,691.000000,0.000000,637.000000,596.000000,691.000000,0.504849,3.000000,0.000000,1.500000,0.000000,0.000000,0.000000
6,0.018700,4.500000,0.000000,792.500000,776.000000,811.000000,0.000000,792.500000,776.000000,811.000000,0.467361,3.000000,0.000000,1.500000,0.000000,0.000000,0.000000


<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [ ]:
text = "What is the sqrt of 101?"

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [ ]:
model.save_lora("grpo_saved_lora")

Verify LoRA is actually trained!

In [ ]:
from safetensors import safe_open

tensors = {}
with safe_open("grpo_saved_lora/adapter_model.safetensors", framework = "pt") as f:
    # Verify both A and B are non zero
    for key in f.keys():
        tensor = f.get_tensor(key)
        n_zeros = (tensor == 0).sum() / tensor.numel()
        assert(n_zeros.item() != tensor.numel())

Now we load the LoRA and test:

In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": "What is the sqrt of 101?"},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    tokenize = False,
)
from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 2048,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False:
    model.save_pretrained("model")
    tokenizer.save_pretrained("model")
if False:
    model.push_to_hub("hf/model", token = "")
    tokenizer.push_to_hub("hf/model", token = "")


### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
